In [27]:
import os
import re
import string
import json

In [46]:
LABEL_DRUG_PROTEIN = 'DRUG-PROTEIN'
LABEL_CHEMICAL = 'CHEMICAL'
LABEL_DISEASE = 'DISEASE'
LABEL_SPECIES = 'SPECIES'

LABEL_PATH = {
    LABEL_DRUG_PROTEIN: ['BC2GM', 'JNLPBA'],
    LABEL_CHEMICAL: ['BC4CHEMD','BC5CDR-chem'],
    LABEL_DISEASE: ['BC5CDR-disease', 'NCBI-disease'],    
    LABEL_SPECIES: ['linnaeus', 's800']
}

DATASET_PATH = 'NER-Data'

FILE_TYPES = ["train.tsv", "devel.tsv", "test.tsv"]



In [52]:
for label in LABEL_PATH:
    for dir_dataset in LABEL_PATH[label]:
        for dataset_file in FILE_TYPES:
            dataset_ner_file = os.path.join(DATASET_PATH, dir_dataset, dataset_file)
            with open(dataset_ner_file) as f_ner: 
                sentenca = ""
                entities = []
                lista_sentenca = []
                ini_entity_atual = -1
                pos_atual = 0
                entidade_atual = ""

                for linha in f_ner:
                    if linha != "\n":
                        if linha[0:linha.find("\t")] not in "!%\')*+,-./:;<=>?@\\]_`|}~" and len(sentenca) > 0 and sentenca[-1] not in "([.-_/\\|<=>+":
                            pos_atual += 1
                            sentenca += " "
                            if len(entidade_atual) > 0:
                                entidade_atual += " "
                            
                        if ("\tO" in linha):
                            if len(entidade_atual) > 0:
                                entities.append({"entidade":entidade_atual.strip(), 
                                                 "start":ini_entity_atual, 
                                                 "end": ini_entity_atual + len(entidade_atual),
                                                 "label": label_atual
                                                })
                                entidade_atual = ""
                                ini_entity_atual = -1

                            linha_tratada = linha.replace("\tO","").replace("\n", "")

                        elif("\tB" in linha):
                            if len(entidade_atual) > 0:
                                entities.append({"entidade":entidade_atual.strip(), "start":ini_entity_atual, "end": ini_entity_atual + len(entidade_atual)})
                            ini_entity_atual = pos_atual
                            linha_tratada = linha.replace("\tB","").replace("\n", "")
                            entidade_atual = linha_tratada                
                        elif("\tI" in linha):
                            linha_tratada = linha.replace("\tI","").replace("\n", "")
                            entidade_atual += linha_tratada

                        pos_atual += len(linha_tratada)
                        sentenca = sentenca + linha_tratada
                    elif linha == "\n":
                        lista_sentenca.append({"texto": sentenca, "entities": entities})
                        sentenca = ""
                        entities=[]


            with open(dataset_ner_file.replace("tsv", "json"), 'w') as json_file:            
                json.dump(lista_sentenca, json_file)
      
    

In [49]:
lista_sentenca

[{'texto': 'Streptococcus ursoris sp.nov., isolated from the oral cavities of bears.',
  'entities': [{'entidade': 'Streptococcusursorissp.nov.',
    'start': 0,
    'end': 27,
    'label': 'DRUG-PROTEIN'}]},
 {'texto': 'Three Gram-positive, catalase-negative, coccus-shaped organisms were isolated from the oral cavities of bears.',
  'entities': []},
 {'texto': 'The isolates were tentatively identified as a streptococcal species based on the results of biochemical tests.',
  'entities': []},
 {'texto': 'Comparative 16S rRNA gene sequencing studies confirmed that the organisms were members of the genus Streptococcus, but they did not correspond to any recognized species of the genus.',
  'entities': []},
 {'texto': 'The nearest phylogenetic relative of the new isolates was Streptococcus ratti ATCC 19645 (T) (98.6%), however, DNA-DNA hybridization analysis showed that the isolates displayed less than 15% DNA-DNA relatedness with the type strain of S.ratti.',
  'entities': [{'entidade': '

In [9]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'